In [1]:
import numpy as np
import pandas as pd
from keras.models import Model, load_model

Using Theano backend.


# Load candidate models

In [5]:
submission_1 = "tests/fixtures/20200731_predictor.h5"
submission_2 = "tests/fixtures/20200730_predictor.h5"
submission_3 = "tests/fixtures/20200729_predictor.h5"

In [6]:
from keras.constraints import Constraint

class Positive(Constraint):

    def __call__(self, w):
        return K.abs(w)

In [7]:
model_1 = load_model(submission_1, custom_objects={"Positive": Positive})
model_2 = load_model(submission_2, custom_objects={"Positive": Positive})
model_3 = load_model(submission_3, custom_objects={"Positive": Positive})

# Load data

## Latest data

In [8]:
# DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_URL = "tests/fixtures/OxCGRT_latest.csv"
latest_df = pd.read_csv(DATA_URL,
                        parse_dates=['Date'],
                        encoding="ISO-8859-1",
                        error_bad_lines=False)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# Handle regions
latest_df["RegionName"].fillna('', inplace=True)
# Replace CountryName by CountryName / RegionName
# np.where usage: if A then B else C
latest_df["CountryName"] = np.where(latest_df["RegionName"] == '',
                                    latest_df["CountryName"],
                                    latest_df["CountryName"] + ' / ' + latest_df["RegionName"])

## Latest data before submission

In [22]:
submission_date = np.datetime64("2020-07-31")

In [23]:
latest_df[latest_df.CountryName == "United States"].Date.max()

Timestamp('2020-08-28 00:00:00')

In [24]:
snapshot_df = latest_df[latest_df.Date <= submission_date]

In [25]:
snapshot_df[snapshot_df.CountryName == "United States"].Date.max()

Timestamp('2020-07-31 00:00:00')

## NPIs since submission

In [26]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing']

In [31]:
npis_df = latest_df[latest_df.Date > submission_date]

In [28]:
npis_df[npis_df.CountryName == "United States"].Date.min()

Timestamp('2020-08-01 00:00:00')

In [29]:
npis_df = npis_df[["CountryName", "Date"] + NPI_COLUMNS]

In [30]:
npis_df[npis_df.CountryName == "United States"].head(30)

,CountryName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
43352,United States,2020-08-01,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43353,United States,2020-08-02,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43354,United States,2020-08-03,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43355,United States,2020-08-04,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43356,United States,2020-08-05,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43357,United States,2020-08-06,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43358,United States,2020-08-07,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43359,United States,2020-08-08,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43360,United States,2020-08-09,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43361,United States,2020-08-10,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0


# Ranking

## Day 1

### NPIs

In [69]:
start_date = np.datetime64("2020-08-01")
start_date

numpy.datetime64('2020-08-01')

In [70]:
end_date = start_date + np.timedelta64(3, 'D')
end_date

numpy.datetime64('2020-08-04')

In [71]:
day_1_npis_df = latest_df[(latest_df.Date >= start_date) & (latest_df.Date <= end_date)]

In [72]:
day_1_npis_df.Date.unique()

array(['2020-08-01T00:00:00.000000000', '2020-08-02T00:00:00.000000000',
       '2020-08-03T00:00:00.000000000', '2020-08-04T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [73]:
# day_1_npis_df = npis_df[npis_df.Date <= day1]

In [74]:
day_1_npis_df[day_1_npis_df.CountryName == "Aruba"][["CountryName", "RegionName", "Date", "ConfirmedCases"] + NPI_COLUMNS].head(30)

,CountryName,RegionName,Date,ConfirmedCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
213,Aruba,,2020-08-01,121.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
214,Aruba,,2020-08-02,121.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
215,Aruba,,2020-08-03,122.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
216,Aruba,,2020-08-04,124.0,0.0,1.0,0.0,4.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0


### Prediction

In [ ]:
preds = model_1.predict([context_input, action_input])

In [109]:
preds_27_df = pd.read_csv("/Users/m_754337/workspace/esp-demo/xprize/tests/fixtures/20200727_predictions.csv",
                          parse_dates=['Date'],
                          encoding="ISO-8859-1",
                          error_bad_lines=False)

In [110]:
preds_27_df["PredictorName"] = "20200727_predictor"

In [111]:
preds_27_df.head()

,CountryName,Date,PredictedDailyNewCases,PredictorName
0,Aruba,2020-08-01,0.789840,20200727_predictor
1,Aruba,2020-08-02,0.857690,20200727_predictor
2,Aruba,2020-08-03,0.000000,20200727_predictor
3,Aruba,2020-08-04,0.000000,20200727_predictor
4,Afghanistan,2020-08-01,73.132138,20200727_predictor


In [112]:
preds_30_df = pd.read_csv("/Users/m_754337/workspace/esp-demo/xprize/tests/fixtures/20200730_predictions.csv",
                          parse_dates=['Date'],
                          encoding="ISO-8859-1",
                          error_bad_lines=False)

In [113]:
preds_30_df["PredictorName"] = "20200730_predictor"

In [114]:
preds_30_df.head()

,CountryName,Date,PredictedDailyNewCases,PredictorName
0,Aruba,2020-08-01,0.710083,20200730_predictor
1,Aruba,2020-08-02,0.823041,20200730_predictor
2,Aruba,2020-08-03,0.000000,20200730_predictor
3,Aruba,2020-08-04,0.000000,20200730_predictor
4,Afghanistan,2020-08-01,71.030063,20200730_predictor


In [115]:
preds_31_df = pd.read_csv("/Users/m_754337/workspace/esp-demo/xprize/tests/fixtures/20200731_predictions.csv",
                          parse_dates=['Date'],
                          encoding="ISO-8859-1",
                          error_bad_lines=False)

In [116]:
preds_31_df["PredictorName"] = "20200731_predictor"

In [117]:
preds_31_df.head()

,CountryName,Date,PredictedDailyNewCases,PredictorName
0,Aruba,2020-08-01,0.647746,20200731_predictor
1,Aruba,2020-08-02,0.792283,20200731_predictor
2,Aruba,2020-08-03,0.000000,20200731_predictor
3,Aruba,2020-08-04,0.000000,20200731_predictor
4,Afghanistan,2020-08-01,72.844680,20200731_predictor


### Actual data

In [85]:
actual_df = latest_df[["CountryName", "RegionName", "Date", "ConfirmedCases"]]

In [87]:
actual_df['ActualDailyNewCases'] = actual_df.groupby('CountryName').ConfirmedCases.diff().fillna(0)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
actual_df = actual_df[(actual_df.Date >= start_date) & (actual_df.Date <= end_date)]

In [100]:
actual_df = actual_df[["CountryName", "RegionName", "Date", "ActualDailyNewCases"]]

In [101]:
actual_df.head()

,CountryName,RegionName,Date,ActualDailyNewCases
213,Aruba,,2020-08-01,1.0
214,Aruba,,2020-08-02,0.0
215,Aruba,,2020-08-03,1.0
216,Aruba,,2020-08-04,2.0
454,Afghanistan,,2020-08-01,168.0


### Actual + Predicted data

In [129]:
ranking_27_df = actual_df.merge(preds_27_df, on=['CountryName', 'Date'], how='left')

In [130]:
ranking_30_df = actual_df.merge(preds_30_df, on=['CountryName', 'Date'], how='left')

In [131]:
ranking_31_df = actual_df.merge(preds_31_df, on=['CountryName', 'Date'], how='left')

In [132]:
ranking_df = ranking_27_df.append(ranking_30_df)

In [133]:
ranking_df = ranking_df.append(ranking_31_df)

In [ ]:
ranking_df.sort_values(by=["CountryName","RegionName","Date","PredictorName"], inplace=True)

In [145]:
ranking_df.head()

,CountryName,RegionName,Date,ActualDailyNewCases,PredictedDailyNewCases,PredictorName,Diff
4,Afghanistan,,2020-08-01,168.0,73.132138,20200727_predictor,94.867862
4,Afghanistan,,2020-08-01,168.0,71.030063,20200730_predictor,96.969937
4,Afghanistan,,2020-08-01,168.0,72.844680,20200731_predictor,95.155320
5,Afghanistan,,2020-08-02,0.0,88.111154,20200727_predictor,88.111154
5,Afghanistan,,2020-08-02,0.0,83.300741,20200730_predictor,83.300741


In [135]:
ranking_df[(ranking_df.CountryName == "United States") &
           (ranking_df.Date == '2020-08-01')]

,CountryName,RegionName,Date,ActualDailyNewCases,PredictedDailyNewCases,PredictorName
716,United States,,2020-08-01,67023.0,77886.825161,20200727_predictor
716,United States,,2020-08-01,67023.0,75688.520235,20200730_predictor
716,United States,,2020-08-01,67023.0,75965.313708,20200731_predictor


In [ ]:
# ranking_df.to_csv("/Users/m_754337/workspace/esp-demo/xprize/tests/fixtures/20200727_.csv", index=False)

In [136]:
ranking_df['Diff'] = (ranking_df["ActualDailyNewCases"] - ranking_df["PredictedDailyNewCases"]).abs()

In [137]:
ranking_df[(ranking_df.CountryName == "United States") &
           (ranking_df.Date == '2020-08-01')]

,CountryName,RegionName,Date,ActualDailyNewCases,PredictedDailyNewCases,PredictorName,Diff
716,United States,,2020-08-01,67023.0,77886.825161,20200727_predictor,10863.825161
716,United States,,2020-08-01,67023.0,75688.520235,20200730_predictor,8665.520235
716,United States,,2020-08-01,67023.0,75965.313708,20200731_predictor,8942.313708


In [139]:
ranking_df[(ranking_df.CountryName == "United States") &
           (ranking_df.Date == '2020-08-01')].sort_values(by='Diff')

,CountryName,RegionName,Date,ActualDailyNewCases,PredictedDailyNewCases,PredictorName,Diff
716,United States,,2020-08-01,67023.0,75688.520235,20200730_predictor,8665.520235
716,United States,,2020-08-01,67023.0,75965.313708,20200731_predictor,8942.313708
716,United States,,2020-08-01,67023.0,77886.825161,20200727_predictor,10863.825161


In [147]:
ranking_df.to_csv("/Users/m_754337/workspace/esp-demo/xprize/tests/fixtures/ranking.csv", index=False)

In [146]:
ranking_df.sort_values(by=["CountryName","RegionName","Date","Diff"])

,CountryName,RegionName,Date,ActualDailyNewCases,PredictedDailyNewCases,PredictorName,Diff
4,Afghanistan,,2020-08-01,168.0,73.132138,20200727_predictor,94.867862
4,Afghanistan,,2020-08-01,168.0,72.844680,20200731_predictor,95.155320
4,Afghanistan,,2020-08-01,168.0,71.030063,20200730_predictor,96.969937
5,Afghanistan,,2020-08-02,0.0,83.300741,20200730_predictor,83.300741
5,Afghanistan,,2020-08-02,0.0,84.184771,20200731_predictor,84.184771
...,...,...,...,...,...,...,...
962,Zimbabwe,,2020-08-03,262.0,76.308265,20200730_predictor,185.691735
962,Zimbabwe,,2020-08-03,262.0,69.539523,20200731_predictor,192.460477
963,Zimbabwe,,2020-08-04,154.0,189.594721,20200731_predictor,35.594721
963,Zimbabwe,,2020-08-04,154.0,195.981371,20200730_predictor,41.981371


In [148]:
# best predictor by sum of diff

In [151]:
ranking_df.groupby('PredictorName').Diff.sum().sort_values()

PredictorName
20200730_predictor    240955.794200
20200731_predictor    245137.073892
20200727_predictor    276667.592976
Name: Diff, dtype: float64